In [ ]:
import os
from os.path import join
import torch
from tqdm import tqdm
from options.train_options import TestOptions
from models import create_model
from util import util
import torchvision.transforms as transforms

# Используем первую доступную GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.backends.cudnn.benchmark = True

if __name__ == '__main__':
    # Загружаем параметры тестирования
    opt = TestOptions().parse()
    
    # Создаём папку для сохранения результатов, если её нет
    save_img_path = opt.results_img_dir
    os.makedirs(save_img_path, exist_ok=True)
    
    # Указываем размер batch'а (должен быть 1 для обработки изображений по одному)
    opt.batch_size = 1

    # Загружаем тестовый датасет
    from datasets import FullColorizationDataset  # Используем обычный датасет, а не fusion
    dataset = FullColorizationDataset(opt)
    dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=2)

    print(f'# Testing images = {len(dataset)}')

    # Создаём модель и загружаем веса первого этапа (Full Colorization)
    model = create_model(opt)
    model.setup_to_test('coco_full')

    # Проходим по датасету и раскрашиваем изображения
    for data_raw in tqdm(dataset_loader, dynamic_ncols=True):
        img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
        model.set_input(img_data)
        model.forward()
        model.save_current_imgs(join(save_img_path, data_raw['file_id'][0] + '.png'))

    print(f'Colorized images saved in {save_img_path}')
